# Power BI Visualization Covid-19 Global Cases Data  

In [1]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd      # Importing Pabdas Library.
import numpy as np    # Importing Numpy Library.
import matplotlib.pyplot as plt   # Importing Mamatplotlib.pyplot Library.
import seaborn as sns   # Importing seaborn Library.
import wget

In [3]:
urls = [
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
]
[wget.download(url) for url in urls]

100% [............................................................................] 980835 / 980835

['time_series_covid19_confirmed_global (3).csv',
 'time_series_covid19_deaths_global (3).csv',
 'time_series_covid19_recovered_global (3).csv']

In [4]:
confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [5]:
dates = confirmed_df.columns[4:]
confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

In [6]:
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

In [7]:
# Merging confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [8]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
308936,NaN,West Bank and Gaza,31.952200,35.233200,12/25/22,703228,5708,0.0
308937,NaN,Winter Olympics 2022,39.904200,116.407400,12/25/22,535,0,0.0
308938,NaN,Yemen,15.552727,48.516388,12/25/22,11945,2159,0.0
308939,NaN,Zambia,-13.133897,27.849332,12/25/22,334021,4019,0.0


In [9]:
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [10]:
full_table.isna().sum()

Province/State    211662
Country/Region         0
Lat                 2138
Long                2138
Date                   0
Confirmed              0
Deaths                 0
Recovered          22449
dtype: int64

In [11]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)

In [12]:
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

In [13]:
full_table = full_table[~(ship_rows)]

In [14]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

C:\Users\Sabae\AppData\Local\Temp\ipykernel_12336\2634332054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']


In [15]:
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

C:\Users\Sabae\AppData\Local\Temp\ipykernel_12336\1890584991.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


In [16]:
# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])
# filling na with 0
full_grouped = full_grouped.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')
# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

C:\Users\Sabae\AppData\Local\Temp\ipykernel_12336\103805664.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']


In [54]:
# df = pd.DataFrame(data, columns=['Country','Recovered'])

In [53]:
# data = [['India', 951], ['Brazil', 754], ['US', 497],['Russia',220],['Colombia',118],['Argentina',92],['Peru',89],['Mexico',85],['South Africa',56],['Iran',47],['Turkey',39],['Germany',27],['Italy',19],['Chile',8],['Saudi Arabia',6]]

In [56]:
# df.to_excel('Country_Recovered_Data.xlsx')

In [ ]:
# full_grouped.to_csv('COVID-19_Global_Data.csv')             # Loading Data With the Help of Pandas.

In [ ]:
# full_grouped.to_excel('COVID-19_Global_Data.xlsx')           # Loading into Excel.